# EO-Lab Tutorial: Search for EnMAP Data (Time-Series Metrics)

This Jupyter notebook demonstrates how to **search the EOC Geoservice STAC catalog** for EnMAP data (L0, L2A) in a given area/time range. It prints:

- **Number of scenes found** in each collection
- **Earliest and latest** acquisition dates
- **Basic time-series metrics** (cloud-cover statistics, time gaps, etc.)
- **Detailed item-by-item listing** with dataset name (STAC item.id), acquisition date, cloud cover, and gap to the previous item.

This notebook **does not download** any data—it simply queries and summarizes availability.

> **Note**: The code cells below are divided similarly to your template. **Please do not modify the Python code** inside them if you wish to preserve the exact functionality. You can edit the bounding box, date range, or other user-configurable parameters, but the code logic remains the same.

## Dependencies and Important Notes

- **Python 3.x** is required.
- You must have installed:
  - `pystac_client` (e.g., `pip install pystac_client`)
  - `requests`
  - (Optional) `shapely` if you do geometry manipulations (though not strictly required here)
- The script leverages the open [EOC Geoservice STAC endpoint](https://geoservice.dlr.de/eoc/ogc/stac/v1/), so **no login or cookie** is required for searching.

In the following cells, you'll see how to specify **time range** and **location** (bounding box or center coordinate), choose **which EnMAP collections** to query, and then run the search to see **basic availability**.

## 1. Imports and Definitions

In this cell, we import all necessary libraries and define helper functionality.

In [ ]:
'''
******************************************************************************
 EnMAP Data Availability (Search-Only) Tutorial
******************************************************************************
This script demonstrates how to:
  1) Search the EOC Geoservice STAC catalog for EnMAP data 
     (L0, L2A) in a given area/time range.
  2) Print basic availability stats (number of scenes, date range).
  3) Compute simple time-series metrics like cloud-cover statistics
     and time gaps between acquisitions.
  4) Provide an item-by-item overview with dataset name (STAC item.id),
     acquisition date, cloud cover, and gap to previous item.

Important: This script does NOT perform downloads. It only queries
the STAC catalog and prints summary information.

Dependencies:
  - Python 3.x
  - pystac_client (pip install pystac_client)
  - requests
  - shapely (optional, if you want geometry manipulations)

Usage:
  1) Adjust the "USER CONFIGURATION" section below.
  2) Run: python enmap_search_tutorial.py
******************************************************************************
'''

import math
import requests
from math import cos, radians
from datetime import datetime
from pystac_client import Client


## 2. User Configuration

Specify:
- Which **collections** to query (e.g., `ENMAP_HSI_L2A`, `ENMAP_HSI_L0_QL`).
- The **time range** for searching (`START_DATE`, `END_DATE`).
- Whether to use a bounding box (`BBOX`) or a **center coordinate** + width in km (`USE_CENTER_COORD`).
- A maximum **cloud cover** threshold (optional) to filter L2 items.

In [ ]:
COLLECTIONS_TO_QUERY = [
    "ENMAP_HSI_L0_QL",  # Level-0 quicklook
    "ENMAP_HSI_L2A"     # Level-2 reflectance data (includes cloud cover)
]

START_DATE = "2024-01-01"
END_DATE   = "2024-12-31"

USE_CENTER_COORD = True
CENTER_COORD = {
    "lat": 50.71868778231684,
    "lon": 7.158329088235492,
    "size_km": 3
}
# If you'd rather use a bounding box directly, set USE_CENTER_COORD=False 
# and define BBOX = [west, south, east, north].

MAX_CLOUD_COVER = 10.0
ENABLE_CLOUD_FILTER = True


## 3. Helper Function

A simple utility to **convert** a center coordinate (lat, lon) plus `size_km` into a bounding box.

In [ ]:
def create_bbox_from_center(lat, lon, size_km):
    """
    Create a bounding box [west, south, east, north]
    from a center (lat, lon) and box width/height in km.
    """
    km_per_degree_lat = 111.0
    km_per_degree_lon = 111.0 * cos(radians(lat))
    half_deg_lat = (size_km / 2) / km_per_degree_lat
    half_deg_lon = (size_km / 2) / km_per_degree_lon
    west  = lon - half_deg_lon
    east  = lon + half_deg_lon
    south = lat - half_deg_lat
    north = lat + half_deg_lat
    return [west, south, east, north]

## 4. EnmapSearchTutorial Class

This class contains the logic to:
- **Search** each specified collection in the STAC.
- **Filter** results by bounding box, time range, and optionally by cloud cover.
- Print **time-series metrics** (date range, average gap, average cloud cover, etc.).
- Show **item-by-item** details (dataset ID, date, cloud cover, gap from previous).

In [ ]:
class EnmapSearchTutorial:
    """
    This class queries EnMAP data from the EOC Geoservice STAC catalog
    for a given bounding box and date range, then prints:
      - Number of items found
      - Date range coverage
      - Basic time-series metrics (cloud cover stats, time gaps, etc.)
      - Detailed item-by-item listing with dataset name (item.id), acquisition date,
        cloud cover, and gap to the previous item
    """
    def __init__(self):
        # Initialize STAC client once. No login needed for searching.
        self.catalog = Client.open("https://geoservice.dlr.de/eoc/ogc/stac/v1/")
    
    def search_collections(
        self,
        collections,
        bbox,
        start_date,
        end_date,
        max_cloud_cover=None
    ):
        """
        Searches each collection for items in the specified bounding box/date range.
        Optionally filters by cloud cover if 'eo:cloud_cover' is provided.

        Returns a dict: {collection_name: [list_of_pystac.Items]}
        """
        results = {}
        for coll in collections:
            print(f"\n=== Searching collection: {coll} ===")
            search_params = {
                "collections": [coll],
                "bbox": bbox,
                "datetime": f"{start_date}T00:00:00Z/{end_date}T23:59:59Z"
            }
            search = self.catalog.search(**search_params)
            
            matched = search.matched()
            print(f"  Found {matched} total items for {coll} (raw).")
            
            items = list(search.items())
            print(f"  Retrieved {len(items)} items in the result list.")
            
            # Convert cloud cover to float here, if present:
            for item in items:
                cc = item.properties.get("eo:cloud_cover", None)
                if cc is not None:
                    try:
                        item.properties["eo:cloud_cover"] = float(cc)
                    except ValueError:
                        item.properties["eo:cloud_cover"] = None

            # Now filter items by cloud cover if requested
            if max_cloud_cover is not None:
                filtered = []
                for it in items:
                    cc_val = it.properties.get("eo:cloud_cover", None)
                    # For L0 data, cc_val might be None => we keep them anyway
                    if cc_val is None:
                        filtered.append(it)
                    else:
                        if cc_val <= max_cloud_cover:
                            filtered.append(it)
                print(f"  After cloud cover <= {max_cloud_cover}: {len(filtered)} items remain.")
                items = filtered

            results[coll] = items
        return results
    
    def print_time_series_metrics(self, items_dict):
        """
        For each collection in items_dict ({coll_name: [pystac.Item, ...]}),
        compute and print:
          - How many items
          - Min/Max/Mean cloud cover (if available)
          - Earliest and latest acquisition dates
          - Basic gap analysis (avg/min/max gap in days)
          - Detailed item-by-item listing with dataset name (item.id),
            acquisition date, cloud cover, and gap to the previous item
        """
        for coll, items in items_dict.items():
            print(f"\n=== Time-Series Metrics for {coll} ===")
            if not items:
                print("  No items found. Skipping metrics.")
                continue
            
            # Build a list of (item_id, datetime_obj, cloud_cover)
            scenes = []
            for it in items:
                item_id = it.id  # The dataset name/ID from STAC
                dt_str  = it.properties.get('datetime')
                if dt_str:
                    dt_obj = datetime.fromisoformat(dt_str.replace("Z",""))
                else:
                    dt_obj = None
                cc_val = it.properties.get("eo:cloud_cover", None)
                scenes.append((item_id, dt_obj, cc_val))
            
            # Filter out scenes that have no valid datetime
            scenes = [s for s in scenes if s[1] is not None]
            if not scenes:
                print("  All items missing 'datetime'—cannot compute date-based metrics.")
                continue
            
            # Sort by date
            scenes.sort(key=lambda x: x[1])
            total_items = len(scenes)
            earliest = scenes[0][1]
            latest   = scenes[-1][1]

            print(f"  Total items: {total_items}")
            print(f"  Date range: {earliest.date()} -> {latest.date()}")

            # Compute time gaps (days) if at least 2 items
            if total_items > 1:
                gaps = []
                for i in range(total_items - 1):
                    gap_days = (scenes[i+1][1] - scenes[i][1]).days
                    gaps.append(gap_days)
                avg_gap = sum(gaps)/len(gaps)
                min_gap = min(gaps)
                max_gap = max(gaps)
                print(f"  Mean gap: {avg_gap:.1f} days (min={min_gap}, max={max_gap})")
            else:
                print("  Only 1 item—no gap metrics.")
            
            # Compute cloud-cover stats
            ccs = [s[2] for s in scenes if (s[2] is not None)]
            if ccs:
                avg_cc = sum(ccs)/len(ccs)
                min_cc = min(ccs)
                max_cc = max(ccs)
                print(f"  Avg cloud cover: {avg_cc:.2f}% (min={min_cc}%, max={max_cc}%)")
            else:
                print("  No cloud cover property (likely L0 or missing data).")

            # Print detailed list for each item (time-series style)
            print("\n  Detailed Items (chronological):")
            for idx, (item_id, dt_obj, cc_val) in enumerate(scenes):
                if idx == 0:
                    gap_str = "N/A (first item)"
                else:
                    gap_days = (dt_obj - scenes[idx-1][1]).days
                    gap_str = f"{gap_days} day{'s' if gap_days != 1 else ''} since previous"

                if cc_val is not None:
                    cc_str = f"{cc_val:.1f}%"
                else:
                    cc_str = "N/A"
                
                print(f"   {idx+1}. Dataset='{item_id}' | Date={dt_obj.date()} | CloudCover={cc_str} | Gap={gap_str}")


## 5. Main Execution

This final cell:
1. Creates an instance of `EnmapSearchTutorial`.
2. Derives the bounding box from either `CENTER_COORD` or a direct `BBOX`.
3. Searches the specified **collections**.
4. Prints **time-series metrics**.

In [ ]:
if __name__ == "__main__":
    # 1) Determine the bounding box
    if USE_CENTER_COORD:
        bbox = create_bbox_from_center(
            CENTER_COORD["lat"], 
            CENTER_COORD["lon"], 
            CENTER_COORD["size_km"]
        )
    else:
        # Example bounding box if not using center coords
        BBOX = [11.230259, 48.051808, 11.337891, 48.117059]
        bbox = BBOX

    # 2) Create the search object
    tutorial = EnmapSearchTutorial()

    # 3) Search for each collection
    max_cc_filter = MAX_CLOUD_COVER if (ENABLE_CLOUD_FILTER and MAX_CLOUD_COVER is not None) else None
    found_items = tutorial.search_collections(
        collections=COLLECTIONS_TO_QUERY,
        bbox=bbox,
        start_date=START_DATE,
        end_date=END_DATE,
        max_cloud_cover=max_cc_filter
    )

    # 4) Print time-series metrics (including item-by-item details)
    tutorial.print_time_series_metrics(found_items)

    print("\nDone! This script searched EnMAP data and reported availability + extended time-series details.")
